In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

import pandas as pd 

# importing functions
import sys
sys.path.append('/Users/audreyburggraf/Desktop/THESIS/Functions')

from functions import *
from unit_conversion_functions import *
from fitting_functions import *
from printing_functions import *

In [2]:
# names 
# inj_params_0P: array holding the injected parameters for the 0 planet fit 
# inj_params_1P: array holding the injected parameters for the 1 planet fit 

In [3]:
df = pd.read_csv('/Users/audreyburggraf/Desktop/THESIS/Gaia Data Files/gaia_data.csv')

#  add the absolute gaia K band magnitude and stellar mass
df["M_ks"] = df.ks_m + 5 + 5*np.log10(df.parallax/1000)

# cut off certain range 
df = df[(4 < df.M_ks) & (df.M_ks < 11)].reset_index(drop=True)

# set the stellar mass 
df["stellar_mass"] =  find_star_mass(df.M_ks)

In [4]:
N1 = 70
N2 = 70000

seed = 8

# call function
parameter_result, synthetic_result, model_result, error_result = find_signal_components(df, N1, N2, print_params='both')

# break down the result statement 

# parameter_result includes the no planet and 1 planet parameters in their proper units 
(inj_params_0P, inj_params_1P) = parameter_result

# model_results includes the signal components and times for the synthetic in [uas] and [years]
(prop_ra_synthetic, prop_dec_synthetic, 
 parallax_ra_synthetic, parallax_dec_synthetic, 
 planetary_ra_synthetic, planetary_dec_synthetic, 
 times_synthetic) = synthetic_result

# model_results includes the signal components and times for the model in [uas] and [years]
(prop_ra_model, prop_dec_model, 
 parallax_ra_model, parallax_dec_model, 
 planetary_ra_model, planetary_dec_model, 
 times_model) = model_result

# error_result includes noise in ra and dec direction plus error (same for ra and dec) in [uas]
(noise_ra, noise_dec, errors) = error_result

Planetary parameters:
e           : 0.13003744369330628  [unitless]
omega       : 5.813494981781561  [radians]
Omega       : 3.9104750247099913  [radians]
cos_i       : 0.48417338181696956  [unitless]
m_planet    : 0.34603994707664776  [Jupiter masses]
log10(P_orb): 0.03807756303253029  [log10(years)]
t_peri      : 0.6801331664876508  [years]
 
Gaia parameters:
alpha0      : 1  [degrees]
delta0      : 2  [degrees]
mu_alpha    : -2.897642821950277  [mas/year]
mu_delta    : -12.29304229257255  [mas/year]
parallax    : 50  [mas]
m_star      : 0.2203081678177399  [M_sun]
x =  149186


In [5]:
signal_ra_obs  = prop_ra_synthetic + parallax_ra_synthetic + planetary_ra_synthetic + noise_ra
signal_dec_obs = prop_dec_synthetic + parallax_dec_synthetic + planetary_dec_synthetic + noise_dec

In [6]:
fitted_params_0P, np_chi_sq = no_planet_fit(inj_params_0P, signal_ra_obs, signal_dec_obs, noise_ra, noise_dec, times_synthetic)

print_parameter_differences(inj_params_0P, fitted_params_0P, return_type='np')

alpha0  : real=1.00000, fitted=1.04117, difference=0.04117
delta0  : real=2.00000, fitted=1.99116, difference=0.00884
mu_alpha: real=-2.89764, fitted=-2.89658, difference=0.00106
mu_delta: real=-12.29304, fitted=-12.28917, difference=0.00388
parallax: real=50.00000, fitted=49.99530, difference=0.00470


In [7]:
fitted_params_1P, wp_chi_sq = one_planet_fit(inj_params_1P, signal_ra_obs, signal_dec_obs, noise_ra, noise_dec, times_synthetic)

print_parameter_differences(inj_params_1P, fitted_params_1P, return_type='wp')

alpha0      : real=1.00000, fitted=1.04608, difference=0.04608
delta0      : real=2.00000, fitted=1.99807, difference=0.00193
mu_alpha    : real=-2.89764, fitted=-2.89501, difference=0.00264
mu_delta    : real=-12.29304, fitted=-12.29220, difference=0.00084
parallax    : real=50.00000, fitted=49.99311, difference=0.00689
e           : real=0.13004, fitted=-0.98338, difference=1.11342
omega       : real=5.81349, fitted=4.74747, difference=1.06602
Omega       : real=3.91048, fitted=2.98333, difference=0.92715
cos_i       : real=0.48417, fitted=-0.03841, difference=0.52259
m_planet    : real=0.34604, fitted=0.77572, difference=0.42968
log10(P_orb): real=0.03808, fitted=0.26878, difference=0.23070
t_peri      : real=0.68013, fitted=0.50426, difference=0.17587


In [16]:
detection_result = detection_function(np_chi_sq, wp_chi_sq, inj_params_1P, fitted_params_1P, N1)


print("np_BIC =", detection_result[0])
print("wp_BIC =", detection_result[1])
print("Delta_BIC =", detection_result[2])
print("detection (yes = 1, no=0) =", detection_result[3])
print("conditions satisfied (Delta BIC>20, log(P) within err, mass within err) =", detection_result[4])

np_BIC = 539.9636528315746
wp_BIC = 290.97320773938316
Delta_BIC = 248.9904450921914
detection (yes = 1, no=0) = 0
conditions satisfied (Delta BIC>20, log(P) within err, mass within err) = [1 0 0]


In [17]:
N1 = 70
N2 = 70000

seed = 8

detection_result = multiple_fitting_function(df, N1, N2, 1)
detection_result

(567.8557207772806, 208.9606174454837, 358.89510333179686, 0, array([1, 0, 0]))
run # 0  conditions_satisfied: [1 0 0]


[0]